In [1]:
!pip install spacy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import string # special operations on strings
import spacy # language models
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
elon = pd.read_csv("P:/DS material/DS Assignments/Solved and unsolved assignments/All DS Assignments/All Datasets for assignments/Elon_musk.csv", encoding = 'latin1')
elon

In [ ]:
elon.shape

In [ ]:
elon.info()

In [ ]:
# remove both the leading and the trailing characters
elon = [y.strip() for y in elon['Text']]
elon

In [ ]:
len(elon)

In [ ]:
# removes empty strings, because they are considered in Python as False
elon = [Text for Text in elon if Text] 
elon

In [ ]:
len(elon)

In [ ]:
# Joining the list into one string/text
text = ' '.join(elon)
text

In [ ]:
# Punctuation

no_punc_text = text.translate(str.maketrans('', '', string.punctuation)) 
no_punc_text

In [ ]:
string.punctuation

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
#Tokenization
from nltk.tokenize import word_tokenize

text_tokens = word_tokenize(no_punc_text)
text_tokens

In [ ]:
len(text_tokens)

In [ ]:
text_tokens[0:50]

In [ ]:
nltk.download('stopwords')

In [ ]:
#Remove stopwords
from nltk.corpus import stopwords
my_stop_words = stopwords.words('english')
sw_list = ['\x92','rt','ye','yeah','haha','Yes','U0001F923','I']
my_stop_words.extend(sw_list)
my_stop_words

In [ ]:
# Simplified look of the no_stop_tokens for loop
""" final_words = []
  for word in text_tokens:
   if word in my_stop_words:
     'do nothing'
   else:
     final_words.append(word)"""

no_stop_tokens = [word for word in text_tokens if not word in my_stop_words]
print(no_stop_tokens[0:40])

In [ ]:
#Noramalize the data
lower_words = [x.lower() for x in no_stop_tokens]
print(lower_words[0:25])

In [ ]:
#Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stemmed_tokens = [ps.stem(word) for word in lower_words]
print(stemmed_tokens[0:40])

In [ ]:
!python -m spacy download en_core_web_sm


In [ ]:
# NLP english language model of spacy library
!pip uninstall spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
import spacy
nlp = spacy.load("en_core_web_sm")


In [ ]:
# lemmas being one of them, but mostly POS, which will follow later
doc = nlp(' '.join(no_stop_tokens))
print(doc[0:40])

In [ ]:
lemmas = [token.lemma_ for token in doc]
print(lemmas[0:40])

### Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lemmas)

In [ ]:
X

In [ ]:
vectorizer.vocabulary_

In [ ]:
vectorizer.vocabulary_.keys()

In [ ]:
vectorizer.vocabulary_.values()

In [ ]:
vectorizer.vocabulary_.items()

In [ ]:
pd.DataFrame.from_records([vectorizer.vocabulary_])

In [ ]:
pd.DataFrame.from_records([vectorizer.vocabulary_]).T

In [ ]:
pd.DataFrame.from_records([vectorizer.vocabulary_]).T.sort_values(0, ascending = True)

In [ ]:
print(vectorizer.get_feature_names()[50:100])

In [ ]:
arr = X.toarray()
arr

In [ ]:
arr.shape

In [ ]:
print(arr[50:100])

### Let's see how bigrams and trigrams can be included here

In [ ]:
vectorizer_ngram_range = CountVectorizer(analyzer='word',
                                         ngram_range=(1,3),
                                         max_features = 100)

bow_matrix_ngram = vectorizer_ngram_range.fit_transform(elon)
bow_matrix_ngram

In [ ]:
print(vectorizer_ngram_range.get_feature_names())

In [ ]:
print(bow_matrix_ngram.toarray())

### TFidf Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_n_gram_max_features = TfidfVectorizer(norm = 'l2',
                                               analyzer = 'word',
                                               ngram_range = (1,3),
                                               max_features = 10)

tf_idf_matrix_n_gram_max_features = vectorizer_n_gram_max_features.fit_transform(elon)
tf_idf_matrix_n_gram_max_features

In [ ]:
print(vectorizer_n_gram_max_features.get_feature_names())

In [ ]:
print(tf_idf_matrix_n_gram_max_features.toarray())

In [ ]:
pd.DataFrame(tf_idf_matrix_n_gram_max_features.toarray(),
            columns = vectorizer_n_gram_max_features.get_feature_names())

### Generate WordCloud

In [ ]:
# Define a function to plot word cloud

def plot_cloud(wordcloud):
    
    # Set figure size
    plt.figure(figsize=(60, 40))

    # Display image
    plt.imshow(wordcloud) 
    
    # No axis details
    plt.axis("off")

In [ ]:
stopwords = STOPWORDS
stopwords.add('will')

In [ ]:
# Generate wordcloud
wordcloud = WordCloud(width = 3000, 
                      height = 2000,
                      background_color='black',
                      max_words=100,
                      colormap='Set2',
                      stopwords=stopwords).generate(text)
# Plot    
plot_cloud(wordcloud)

In [ ]:
# Save image
wordcloud.to_file("wordcloud.png")

In [ ]:
from nltk import tokenize

In [ ]:
sentences=tokenize.sent_tokenize(' '.join(elon))
sentences

In [ ]:
sent_df=pd.DataFrame(sentences,columns=['sentence'])
sent_df

### Named Entity Recognition

In [ ]:
# Parts Of Speech (POS) Tagging
nlp = spacy.load('en_core_web_sm')

one_block=text
doc_block=nlp(one_block)
spacy.displacy.render(doc_block,style='ent',jupyter=True)

In [ ]:
for token in doc_block[100:200]:
    print(token,token.pos_) 

In [ ]:
# Filtering the nouns and verbs only
nouns_verbs=[token.text for token in doc_block if token.pos_ in ('NOUN','VERB')]
print(nouns_verbs[100:200])

In [ ]:
# Counting the noun & verb tokens
cv=CountVectorizer()

X=cv.fit_transform(nouns_verbs)
sum_words=X.sum(axis=0)

words_freq=[(word,sum_words[0,idx]) for word,idx in cv.vocabulary_.items()]
words_freq=sorted(words_freq, key=lambda x: x[1], reverse=True)

wd_df=pd.DataFrame(words_freq)
wd_df.columns=['word','count']

In [ ]:
# viewing top ten results
wd_df[0:10]

In [ ]:
# Visualizing results (Barchart for top 10 nouns + verbs)
wd_df[0:10].plot.bar(x='word', figsize=(20,7), title='Top 10 nouns and verbs',color='red');

### Emotion Mining Analysis

In [ ]:
from nltk import tokenize

In [ ]:
sentences=tokenize.sent_tokenize(' '.join(elon))
sentences

In [ ]:
sent_df=pd.DataFrame(sentences,columns=['sentence'])
sent_df

In [ ]:
# Emotion Lexicon - Affin
affin=pd.read_csv("Afinn.csv",sep=',',encoding='Latin-1')
affin

In [ ]:
affinity_scores=affin.set_index('word')['value'].to_dict()
affinity_scores

In [ ]:
# Custom function: score each word in a sentence in lemmatised form, but calculate the score for the whole original sentence
nlp=spacy.load('en_core_web_sm')
sentiment_lexicon=affinity_scores

def calculate_sentiment(text:str=None):
    sent_score=0
    if text:
        sentence=nlp(text)
        for word in sentence:
            sent_score+=sentiment_lexicon.get(word.lemma_,0)
    return sent_score

In [ ]:
# manual testing
calculate_sentiment(text='great')

In [ ]:
# Calculating sentiment value for each sentence
sent_df['sentiment_value']=sent_df['sentence'].apply(calculate_sentiment)
sent_df['sentiment_value']

In [ ]:
# how many words are there in a sentence
sent_df['word_count']=sent_df['sentence'].str.split().apply(len)
sent_df['word_count']

In [ ]:
sent_df.sort_values(by='sentiment_value')

In [ ]:
# negative sentiment score of the whole review
sent_df[sent_df['sentiment_value']<=0]

In [ ]:
# positive sentiment score of the whole review
sent_df[sent_df['sentiment_value']>0]

In [ ]:
# Adding index cloumn
sent_df['index']=range(0,len(sent_df))
sent_df

In [ ]:
# Plotting the sentiment value for whole review
plt.figure(figsize=(20,8))
sns.distplot(sent_df['sentiment_value'],color='red')

In [ ]:
# Correlation analysis
sent_df.plot.scatter(x='word_count',y='sentiment_value',title='Sentence sentiment value to sentence word count',figsize=(20,8),color='red')

In [ ]:
# density plot
ax=plt.axes()
ax.set_facecolor("black")
sent_df['sentiment_value'].plot(kind='kde',figsize=(10,5),color='red',linewidth=3)

In [ ]:
# density plot
ax=plt.axes()
ax.set_facecolor("black")
sent_df['word_count'].plot(kind='kde',figsize=(10,5),color='red',linewidth=3)

In [ ]:
# Plotting the line plot for sentiment value of whole review
plt.figure(figsize=(20,10))
sns.lineplot(y='sentiment_value',x='index',data=sent_df,color='red')

In [ ]:
sent_df['Sentiment_Class'] = pd.cut(x=sent_df['sentiment_value'],bins=[-8, -1, 0, 17], 
                                    labels=['Negative','Neutral','Positive'], right = True)

In [ ]:
sns.countplot(x = 'Sentiment_Class', data = sent_df)

In [ ]:
sent_df['Sentiment_Class'].value_counts()

## Q.2 
### E-commerce review 

In [ ]:
# Importing relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import requests
import string

from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup as bs

In [ ]:
samsung_reviews = []

In [ ]:
for i in range (1,31):
    op=[]
    url ="https://www.amazon.in/product-reviews/B08444S68Q/ref=acr_dp_hist_5?ie=UTF8&filterByStar=five_star&reviewerType=all_reviews"+str(i)
    response = requests.get(url)
    soup = bs(response.content,"html.parser")
    reviews = soup.findAll("span",attrs = {"class","a-size-base review-text review-text-content"})
    for i in range(len(reviews)):
        op.append(reviews[i].text)    
    samsung_reviews = samsung_reviews+op

In [ ]:
samsung_reviews

In [ ]:
reviews = ' '.join(samsung_reviews)

### Data Cleaning

In [ ]:
reviews = re.sub("[^A-Za-z" "]+"," ",reviews).lower() 
reviews = re.sub("[0-9" "]+"," ",reviews).lower() 
reviews = re.sub(r'^https?:\/\/.*[\r\n]*', '', reviews).lower()

### Tokenize

In [ ]:
reviews_token = word_tokenize(reviews)
reviews_without_sw = [word for word in reviews_token if not word in stopwords.words()]

In [ ]:
reviews_without_sw

In [ ]:
reviews = ' '.join(reviews_without_sw)

### Wordcloud

In [ ]:
def plot_cloud(wordcloud):
    plt.figure(figsize=(40,30))
    plt.imshow(wordcloud)
    plt.axis('off')
    

from wordcloud import WordCloud, STOPWORDS
STOPWORDS.add('refresh')
wordcloud=WordCloud(width=3000,height=2000,background_color='black',max_words=70,
                   colormap='Set1',stopwords=STOPWORDS).generate(reviews)
plot_cloud(wordcloud)

### Emotion Mining

In [ ]:
with open("positive-words.txt", 'r') as f:
    positive=f.read().split("\n")

In [ ]:
positive = positive[35:]

In [ ]:
reviews_pos=' '.join([word for word in reviews_without_sw if word in positive])

In [ ]:
# Word Cloud for positive words

def plot_cloud(wordcloud):
    plt.figure(figsize=(40,30))
    plt.imshow(wordcloud)
    plt.axis('off')
    

from wordcloud import WordCloud, STOPWORDS
wordcloud=WordCloud(width=3000,height=2000,background_color='black',max_words=70,
                   colormap='Set1',stopwords=STOPWORDS).generate(reviews_pos)
plot_cloud(wordcloud)

In [ ]:
with open("negative-words.txt","r", encoding = 'latin1') as f:
    negative = f.read().split("\n")

In [ ]:
negative = negative[35:]

In [ ]:
reviews_neg=' '.join([word for word in reviews_without_sw if word in negative])

In [ ]:
# Word Cloud for negative words
def plot_cloud(wordcloud):
    plt.figure(figsize=(40,30))
    plt.imshow(wordcloud)
    plt.axis('off')
    

from wordcloud import WordCloud, STOPWORDS
wordcloud=WordCloud(width=3000,height=2000,background_color='black',max_words=70,
                   colormap='Set1',stopwords=STOPWORDS).generate(reviews_neg)
plot_cloud(wordcloud)